# Processamento dos slides

Primeiramente, a conversão é de PPTX para TXT. É possível pular essa etapa e ir diretamente para JSON, mas a conversão para TXT é útil para visualização e depuração.

In [8]:
from pptx import Presentation
import glob

files = glob.glob("slides_adapt\\*.pptx")
# files = ["slides\\Louvores Avulsos CIAs 2022 - COM ANIMAÇÃO.pptx"]
# files.remove("slides\\01.COLETÂNEA_IGREJAS_2022_TV-16.9.pptx")
# files.remove("slides\\03.COLETÂNEA DE CIAS_2021 TV.pptx")
files

['slides_adapt\\01.COLETANEA_IGREJAS_2022_TV-16.9_ADAPT.pptx',
 'slides_adapt\\03.COLETÂNEA DE CIAS_2021 TV_ADAPT.pptx',
 'slides_adapt\\Louvores Avulsos CIAs 2022 - COM ANIMAÇÃO_ADAPT.pptx',
 'slides_adapt\\LOUVORES AVULSOS_Rev_31.12.22.pptx']

In [9]:
from pptx.enum.shapes import MSO_SHAPE_TYPE

for file in files:
    new_file = "slides_txt\\" + file.split("\\")[1] + ".txt"

    f = open(new_file, "w")

    prs = Presentation(file)
    for i, slide in enumerate(prs.slides):

        f.write(f"\nSLIDE_{i}\n")

        for shape in slide.shapes:
            f.write(f"SHAPE_{shape.shape_type}")

            if shape.shape_type == MSO_SHAPE_TYPE.AUTO_SHAPE:
                f.write(f"_AUTO_SHAPE_{shape.auto_shape_type}\n")
                f.write(f"HEIGHT_{shape.height}\n")
                f.write(f"TOP_{shape.top}\n")

            if hasattr(shape, "text"):
                f.write(f"\nSTART_TEXT\n{shape.text}\nEND_TEXT\n")
            
            f.write("\n")

        for shape in slide.shapes:
            if hasattr(shape, "text"):
                if shape.text.lower() == "índice":
                    f.write("\n__END__\n")

    f.close()

# Processamento do texto

In [10]:
files_txt = glob.glob("slides_txt\\*.txt")
files_txt

['slides_txt\\01.COLETANEA_IGREJAS_2022_TV-16.9_ADAPT.pptx.txt',
 'slides_txt\\03.COLETÂNEA DE CIAS_2021 TV_ADAPT.pptx.txt',
 'slides_txt\\Louvores Avulsos CIAs 2022 - COM ANIMAÇÃO_ADAPT.pptx.txt',
 'slides_txt\\LOUVORES AVULSOS_Rev_31.12.22.pptx.txt']

In [11]:
import re
import json

auto_shape_pattern = r"_AUTO_SHAPE_([A-Z_]+)"
shape_pattern = r"SHAPE_([A-Z_]+)"

for file_txt in files_txt:
    praises = []
    with open(file_txt, "r") as f:
        text = f.read()
    
    for praise in text.split("__END__"):
        praise_struc = {}
        praise_struc["slides"] = []

        for slide in praise.split("SLIDE_"):
            slide_struc = {}
            slide_num = slide.split("\n")[0].strip()
            if not slide_num:
                continue

            slide_struc["slide"] = slide_num
            slide_struc["shapes"] = []

            for shape in slide.split("\nSHAPE_"):
                record_line = False
                text_inside = ""
                shapes_struc = {}

                for line in shape.split("\n"):
                    if "SHAPE_" in line:
                        match = re.search(auto_shape_pattern, line)
                        auto_shape = match.group(1)

                        shapes_struc["auto_shape"] = auto_shape
                        shapes_struc["shape"] = "AUTO_SHAPE"

                    if "HEIGHT_" in line:
                        height = line.split("_")[1]
                        shapes_struc["height"] = int(height)
                    if "TOP_" in line:
                        top = line.split("_")[1]
                        shapes_struc["top"] = int(top)

                    if "END_TEXT" in line:
                        record_line = False

                    if record_line:
                        text_inside += line + "\n"
                    else:
                        if text_inside:
                            shapes_struc["text"] = text_inside.strip()

                    if "START_TEXT" in line:
                        text_inside = ""
                        record_line = True

                if shapes_struc:
                    slide_struc["shapes"].append(shapes_struc)

            praise_struc["slides"].append(slide_struc)

        # pegar nome/numero do hino antes de salvar
        praises.append(praise_struc)

    new_file = "slides_json\\" + file_txt.split("\\")[1] + ".json"
    with open(new_file, "w") as f:
        f.write(json.dumps(praises))


# Processamento do JSON

In [12]:
def get_texts(d):
    if isinstance(d, dict):
        for k, v in d.items():
            if k == "text":
                yield v
            else:
                yield from get_texts(v)
    elif isinstance(d, list):
        for v in d:
            yield from get_texts(v)

In [42]:
import json

with open("slides_json\\01.COLETANEA_IGREJAS_2022_TV-16.9_ADAPT.pptx.txt.json", "r") as f:
    praises = json.load(f)

praises

[{'slides': [{'slide': '0',
    'shapes': [{'auto_shape': 'RECTANGLE',
      'shape': 'AUTO_SHAPE',
      'height': 581728,
      'top': 400249,
      'text': '01 – O SANGUE DE JESUS TEM PODER'},
     {'auto_shape': 'RECTANGLE',
      'shape': 'AUTO_SHAPE',
      'height': 3138423,
      'top': 1959880,
      'text': 'O SANGUE DE JESUS TEM PODER,\nPODER QUE A MIM PODE VALER,\nSE COMUNHÃO NÃO POSSO EU SENTIR, SENHOR,\nDERRAMA DO TEU SANGUE REMIDOR.'}]},
   {'slide': '1',
    'shapes': [{'auto_shape': 'RECTANGLE',
      'shape': 'AUTO_SHAPE',
      'height': 3674724,
      'top': 1383321,
      'text': 'CORO\nDERRAMA DO TEU SANGUE SOBRE MIM, Ó SENHOR,\nDERRAMA DO TEU SANGUE SOBRE MIM, Ó SENHOR,\nSEI QUE ESTÁS AQUI E ISTO QUERO EU SENTIR,\nDERRAMA DO TEU SANGUE SOBRE MIM.'}]},
   {'slide': '2',
    'shapes': [{'auto_shape': 'RECTANGLE',
      'shape': 'AUTO_SHAPE',
      'height': 3138423,
      'top': 1959406,
      'text': 'EM TUA PRESENÇA ESTOU, SENHOR,\nDE TI NECESSITO MUITO AMOR,\nPE

In [43]:
erros_conhecidos = [
    "ABENÇOA-NOS SENHOR, DERRAMA SOBRE NÓS TUA PAZ ABENÇOA-NOS SENHOR, DERRAMA SOBRE NÓS TEU AMOR.",  # indice extra
    "MEU JESUS, SALVADOR,  OUTRO IGUAL NÃO HÁ. TODOS OS DIAS QUERO LOUVAR  AS MARAVILHAS DE TEU AMOR. CONSOLO, ABRIGO,  FORÇA E REFÚGIO É O SENHOR. COM TODO O MEU SER,  COM TUDO O QUE SOU,  SEMPRE TE ADORAREI.",  # pedaco de louvor
    "PODES CLAMAR, PODES CHORAR, EU ESTAREI PRONTO PRA TE AJUDAR, E SE O CORAÇÃO DESFALECER, CONFIA EM MIM SOU JESUS  E TE FAÇO VENCER.",  # pedaco de louvor
    "AO CORDEIRO GLÓRIA E HONRA,  SALVOS NÃO CESSEIS DE DAR. GLÓRIA, HONRA SEMPRE A DEUS ENTOAREI!  AMÉM!",  # indice extra
]
TAGS_LITERAIS = [
    "TODOS",
    "M",
    "H",
    "T",
    "BIS",
    "VARÕES",
    "SERVAS",
]
TAGS_CONTROLE = [
    "ÍNDICE",
    "CORO\n",
    "2X",
    "3X",
    "4X",
    "()",
    "(TODOS)",
    "(M)",
    "(H)",
    "(T)",
    "(BIS)",
    "(VARÕES)",
    "(SERVAS)",
    "REPETIR O LOUVOR",
    "FINAL:",
    "BIS NO FINAL",
    "IGREJA CRISTÃ MARANATA",
    "ATUALIZAÇÃO",
    "\nINSTRUMENTOS",
]

In [58]:
def return_possible_title(texts):
    possible_title = [
        text
        for text in texts
        if all(opt not in text.upper() for opt in TAGS_CONTROLE)
        and text.strip() != ""
        and text.upper() not in TAGS_LITERAIS
    ]
    if possible_title:
        min_string = min(possible_title, key=len)
        title_index = texts.index(min_string)
        title = min_string.upper().replace("–", "-").replace("\n", " ")
        if title not in erros_conhecidos:
            return title, title_index

    return None, None


new_structure = []


def set_text_clean(texts_wo_title):
    texts_clean = [line for line in texts_wo_title if line not in TAGS_LITERAIS]
    new_texts_clean = []
    for line in texts_clean:
        for tag in TAGS_CONTROLE:
            line = line.upper().replace(tag, "")
        line = line.strip()
        if line:
            new_texts_clean.append(line)
    texts_clean = new_texts_clean
    texts_clean = list(dict.fromkeys(texts_clean))

    texts_clean = " ".join(texts_clean)
    texts_clean = texts_clean.replace("\n", " ")
    # remove all double quotes
    texts_clean = texts_clean.replace("“", "")
    texts_clean = texts_clean.replace("”", "")
    texts_clean = texts_clean.replace('"', "")
    return texts_clean


def set_text_full(texts_wo_title):
    texts_full = [
        line
        for line in texts_wo_title
        if line.upper() != "ÍNDICE" and line.strip() != ""
    ]
    texts_full = "\n\n".join(texts_full)
    texts_full = texts_full.replace("\n\nBIS", "\nBIS")
    texts_full = texts_full.replace('"', "'")
    return texts_full


def return_number(title):
    if title:
        for word in title.split(" "):
            match = re.search(r"\d+", word)
            if match:
                return match.group()
    return "null"


for praise in praises:
    texts = list(get_texts(praise))
    if not texts:
        continue

    # clean double or more spaces in string array
    texts = [re.sub(r"\s+", " ", text.replace("–", "-")) for text in texts]

    title, title_index = return_possible_title(texts)
    numero = return_number(title)

    texts_wo_title = texts.copy()
    if title_index:
        texts_wo_title.pop(title_index)

    texts_full = set_text_full(texts_wo_title)
    texts_clean = set_text_clean(texts_wo_title)

    new_structure.append(
        {
            "numero": numero,
            "nome": title,
            "texto": texts_full,
            "texto_limpo": texts_clean,
            "coletanea_id": 1,
        }
    )

new_structure

[{'numero': '01',
  'nome': '01 - O SANGUE DE JESUS TEM PODER',
  'texto': '01 - O SANGUE DE JESUS TEM PODER\n\nO SANGUE DE JESUS TEM PODER, PODER QUE A MIM PODE VALER, SE COMUNHÃO NÃO POSSO EU SENTIR, SENHOR, DERRAMA DO TEU SANGUE REMIDOR.\n\nCORO DERRAMA DO TEU SANGUE SOBRE MIM, Ó SENHOR, DERRAMA DO TEU SANGUE SOBRE MIM, Ó SENHOR, SEI QUE ESTÁS AQUI E ISTO QUERO EU SENTIR, DERRAMA DO TEU SANGUE SOBRE MIM.\n\nEM TUA PRESENÇA ESTOU, SENHOR, DE TI NECESSITO MUITO AMOR, PELO PODER QUE HÁ EM TEU SANGUE REMIDOR, Ó VEM PURIFICAR-ME, MEU SENHOR.\n\nCORO DERRAMA DO TEU SANGUE SOBRE MIM, Ó SENHOR, DERRAMA DO TEU SANGUE SOBRE MIM, Ó SENHOR, SEI QUE ESTÁS AQUI E ISTO QUERO EU SENTIR, DERRAMA DO TEU SANGUE SOBRE MIM.',
  'texto_limpo': '01 - O SANGUE DE JESUS TEM PODER O SANGUE DE JESUS TEM PODER, PODER QUE A MIM PODE VALER, SE COMUNHÃO NÃO POSSO EU SENTIR, SENHOR, DERRAMA DO TEU SANGUE REMIDOR. CORO DERRAMA DO TEU SANGUE SOBRE MIM, Ó SENHOR, DERRAMA DO TEU SANGUE SOBRE MIM, Ó SENHOR, SEI QUE EST

# Criar arquivo de inserção

In [70]:
with open("db\\migrations\\003-hinos-slides.sql", "w") as f:
    for hino in new_structure:
        text = (
            "INSERT INTO hino (numero, nome, texto, texto_limpo, coletanea_id) VALUES ('"
            + hino["numero"]
            + "', '"
            + hino["nome"]
            + "', '"
            + hino["texto"].replace("\n", "\\n").replace("'", "''")
            + "', '"
            + hino["texto_limpo"].replace("'", "''")
            + "',  "
            + str(hino["coletanea_id"])
            + ");\n"
        )
        f.write(text)